In [1]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister
import copy
import warnings
import networkx as nx
import numpy as np
from qiskit import QuantumCircuit, execute
from random import randrange
from Encoder import SurfaceCode

import matplotlib
try:
    from qiskit import Aer
    HAS_AER = True
except ImportError:
    from qiskit import BasicAer
    HAS_AER = False

# Circuit Generation for Surface Codes

Surface Codes are CSS codes which encode logical states in a two dimensional lattice of qubits. Syndromes are measured along all the different squares as $X_1 X_2 Z_1 Z_2$ on each edge. The graph below (also in readme) is a surface code patch of distance d=3 such that number of qubits required is $d^2$ and number of syndrom qubits required is $d^2-1$. Here, blue patches are Z stabilizers and red patches are X stabilizers. Z stabilizers entangle qubits on data qubits (grey) in corners as control, with syndrome qubits (black) in the centre as target. Similarly, X stabilizers entangle qubits with controlled not on data qubits (grey) in corners as target, with syndrome qubits (black) in the centre as control.The Z and N marked in each patch determines the order of CX labeled in black. Grey coordinate labels are data qubit locations and black labels are syndrome qubit locations. The straight line marked Z<sub>L</sub> signifies that a logical Z is applied by operating Z on each qubit, on any horizontal line in the lattice. Similarly, the straight line marked X<sub>L</sub> signifies that a logical X is applied by operating X on each qubit, on any vertical line in the lattice. We choose one convention and say top edge signifies a Z logical operation and left edge signifies X logical operation! 
<img src="Lattice.png">
The code assigns node locations to the lattices using SurfaceCode(d,d).lattice() and then assigns the order of controlled not for each square using SurfaceCode(d,d).connection(). This is then used by the syndrome measurement to apply controlled nots for each square according to the numbers and type of stabilizer the square execute (red: X, blue:Z). 
 Thus, the above circuit is created and each syndrome qubit is measured. This is called syndrome measurement and is repeated T=d times. The results from each syndrome measurement are then processed to extract error nodes i.e. nodes which were flipped in consecutive syndrome measurements. This information is then utilised by the classes in syndrome_graph and fitter.py files to create error graphs and perform matching (explained in section "fitters.py"), to deduce the most probable error. Finally, logical Z error is concluded by checking if there were odd number of qubits with errors on top (Z<sub>L</sub>) edge and logical X error is concluded if there odd number of qubits with errors on the left (X<sub>L</sub>) edge

#   Circuit Execution (Logical 0)


In [5]:
if HAS_AER:
    simulator = Aer.get_backend('qasm_simulator')
else:
    simulator = BasicAer.get_backend('qasm_simulator')
job = execute(SurfaceCode(3,3).circuit['0'], simulator)
raw_results = {}
raw_results['0'] = job.result().get_counts()
processed_results={}
processed_results=SurfaceCode(3,3).process_results(raw_results['0'])
j=0
for i in (processed_results):
    print("results from circuit execution round",j,":", processed_results[2])
    j=j+1
nodesX,nodesZ=SurfaceCode(3,3).extract_nodes(processed_results)
print("error nodes in X", nodesX)
print("error nodes in Z", nodesZ)
print("No Z error as logical 0 state is an eigenstate of logical Z (given no noise is added to the system)")

results from circuit execution round 0 : [0, 0, 0, 0, 0, 0, 0, 0]
results from circuit execution round 1 : [0, 0, 0, 0, 0, 0, 0, 0]
results from circuit execution round 2 : [0, 0, 0, 0, 0, 0, 0, 0]
results from circuit execution round 3 : [0, 0, 0, 0, 0, 0, 0, 0]
error nodes in X [(2, -0.5, 0.5), (2, 1.5, 0.5)]
error nodes in Z []
No Z error as logical 0 state is an eigenstate of logical Z (given no noise is added to the system)


# SurfaceEncoder Circuit

In [6]:
qc=SurfaceCode(3,3).circuit['0']
qc.draw()

┌───┐                               ░                     »
       ancilla_0: ┤ H ├───────────────────────────────░───■─────────────────»
                  ├───┤                               ░   │  ┌───┐          »
       ancilla_1: ┤ X ├───────────────────────────────░───┼──┤ X ├──────────»
                  └─┬─┘┌───┐                          ░   │  └─┬─┘          »
       ancilla_2: ──┼──┤ H ├────────────■─────────────░───┼────┼────■───────»
                    │  ├───┤            │             ░   │    │    │       »
       ancilla_3: ──┼──┤ X ├────────────┼─────────────░───┼────┼────┼───────»
                    │  └─┬─┘            │             ░   │    │    │  ┌───┐»
       ancilla_4: ──┼────┼──────────────┼─────────────░───┼────┼────┼──┤ X ├»
                    │    │  ┌───┐       │             ░   │    │    │  └─┬─┘»
       ancilla_5: ──┼────┼──┤ H ├───────┼────■────────░───┼────┼────┼────┼──»
                    │    │  ├───┤       │    │        ░   │    │    │    │  »
       ancilla_6: ──┼────┼──┤ X ├───────┼────┼────────░───┼────┼────┼────┼──»
                    │    │  └─┬─┘┌───┐  │    │        ░   │    │    │    │  »
       ancilla_7: ──┼────┼────┼──┤ H ├──┼────┼────■───░───┼────┼────┼────┼──»
                    │    │    │  └───┘  │    │    │   ░ ┌─┴─┐  │    │    │  »
          data_0: ──■────┼────┼─────────┼────┼────┼───░─┤ X ├──┼────┼────┼──»
                         │    │       ┌─┴─┐  │    │   ░ └───┘  │    │    │  »
          data_1: ───────┼────┼───────┤ X ├──┼────┼───░────────■────┼────┼──»
                         │    │       └───┘  │    │   ░             │    │  »
          data_2: ───────■────┼──────────────┼────┼───░─────────────┼────┼──»
                              │            ┌─┴─┐  │   ░             │    │  »
          data_3: ────────────┼────────────┤ X ├──┼───░─────────────┼────■──»
                              │            └───┘  │   ░           ┌─┴─┐     »
          data_4: ────────────■───────────────────┼───░───────────┤ X ├─────»
                                                  │   ░           └───┘     »
          data_5: ────────────────────────────────┼───░─────────────────────»
                                                  │   ░                     »
          data_6: ────────────────────────────────┼───░─────────────────────»
                                                ┌─┴─┐ ░                     »
          data_7: ──────────────────────────────┤ X ├─░─────────────────────»
                                                └───┘ ░                     »
          data_8: ────────────────────────────────────░─────────────────────»
                                                      ░                     »
round_0ancilla_0: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_1: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_2: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_3: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_4: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_5: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_6: ══════════════════════════════════════════════════════════»
                                                                            »
round_0ancilla_7: ══════════════════════════════════════════════════════════»
                                                                            »
round_1ancilla_0: ══════════════════════